In [1]:
# Scrape n documents from 2 categories from wikipedia (say sports and education)
# Pre-process and clean all the documents
# Prepare 
# 1. Unigram count matrix
# 2. Bigram probability matrix
# 3. TF-IDF matrix
# Apply appropriate naive bayes classification for each matrix

In [1]:
%pip install nltk
%pip install requests
%pip install beautifulsoup4
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# sports_urls = [
# 	"https://theathletic.com/5275341/2024/02/14/jason-kelce-travis-kelce-super-bowl-andy-reid/",
#  	"https://www.nytimes.com/2024/02/12/business/media/super-bowl-ratings-record.html",
#  	"https://www.nytimes.com/2024/02/12/business/tiger-woods-sun-day-red.html",
#  	"https://theathletic.com/5265696/2024/02/14/there-may-be-no-harry-kane-curse-but-he-will-still-be-feeling-the-pressure/",
#  	"https://theathletic.com/5270421/2024/02/13/manchester-city-copenhagen-de-bruyne-grealish/"
# ]
# education_urls = [
# 	"https://greatergood.berkeley.edu/article/item/six_ways_to_find_your_courage_during_challenging_times",
#  	"https://greatergood.berkeley.edu/article/item/calm_clear_and_kind_what_students_want_from_their_teachers",
#  	"https://greatergood.berkeley.edu/article/item/five_ways_to_support_the_wellbeing_of_school_leaders",
#  	"https://greatergood.berkeley.edu/article/item/how_to_help_your_students_develop_positive_habits",
#  	"https://greatergood.berkeley.edu/article/item/four_ways_to_inspire_humble_curiosity_in_your_students"
# ]

sports_urls = [
  "https://en.wikipedia.org/wiki/Cricket",
  "https://en.wikipedia.org/wiki/Hockey",
  "https://en.wikipedia.org/wiki/Football",
]
education_urls = [
  "https://en.wikipedia.org/wiki/Education",
  "https://en.wikipedia.org/wiki/Knowledge",
  "https://en.wikipedia.org/wiki/Procedural_knowledge",
]

In [4]:
import nltk
import requests
nltk.download('punkt')
from bs4 import BeautifulSoup

REPLACES = {
    '|': '',
    '\'': '',
    '\"': '',
    '\\n': ' ',
    '\\t': ' ',
    '\\s': ' ',
    ',': '',
    '..': '',
    '...': '',
    '--': '',
}

def get_text_corpus(url):
  page = requests.get(url).content
  content = BeautifulSoup(page, 'html.parser')
  text = content.find(attrs={"id": "bodyContent"}).text.replace("\n", " ")
  for key in REPLACES:
    text = text.replace(key, REPLACES[key])
  text = " ".join(text.split())
  return text

[nltk_data] Downloading package punkt to /home/genos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def get_corpus_by_category (url_list):
  text_corpus = [get_text_corpus(url) for url in url_list]
  return " ".join(text_corpus)
    
all_text_corpus = dict()
all_text_corpus["sports"] = get_corpus_by_category(sports_urls)
all_text_corpus["education"] = get_corpus_by_category(education_urls)

In [6]:
def get_words(corpus):
  words = []
  for word in nltk.word_tokenize(corpus):
    if word.isalpha():
      words.append(word)
  return words

unique_words = []
category_wise_words = dict()
for category, content in all_text_corpus.items():
  words = get_words(content)
  category_wise_words[category] = words
  unique_words = unique_words+ words

In [7]:
def get_word_frequency_by_category(arr):
  freq_counter = dict()
  for word in arr:
    if word not in freq_counter:
      freq_counter[word]=1
    else:
      freq_counter[word] +=1
  return freq_counter

education_freq = get_word_frequency_by_category(category_wise_words["education"])
sports_freq = get_word_frequency_by_category(category_wise_words["sports"])   

In [8]:
word_freq_category_wise = dict()

for word in unique_words:
  word_freq_category_wise[word] = { 
    "sports": sports_freq[word] if word in sports_freq else 0, 
    "education": education_freq[word] if word in education_freq else 0 
  }

In [15]:
import pandas as pd
df = pd.DataFrame(word_freq_category_wise).transpose()
df

,sports,education
From,8,16
Wikipedia,3,4
the,1838,1580
free,8,10
encyclopedia,3,3
...,...,...
epistemologyDefinitions,0,1
knowledgePsychological,0,1
conceptsHidden,0,1
publisherArticles,0,1


In [34]:
from nltk import bigrams
from collections import Counter
import numpy as np

def compute_bigram_prob_matrix(texts):
    all_bigrams = [
        bigram for text in texts for bigram in bigrams(text.split())]
    bigram_counts = Counter(all_bigrams)
    bigram_prob_matrix = np.zeros(
        (len(texts), len(bigram_counts)), dtype=float)
    unique_bigrams = sorted(set(all_bigrams))
    for i, _ in enumerate(texts):
        for j, bigram in enumerate(unique_bigrams):
            bigram_prob_matrix[i, j] = bigram_counts[bigram] / len(all_bigrams)
    return pd.DataFrame(bigram_prob_matrix, columns=unique_bigrams)


compute_bigram_prob_matrix([all_text_corpus["sports"]]).transpose()
# .to_csv('filename.csv', encoding='utf-8') # False: not include index


,0
"(&, 11;)",0.000029
"(&, 131.)",0.000029
"(&, 184.)",0.000029
"(&, 404–410.)",0.000029
"(&, 8;)",0.000029
...,...
"(–, were)",0.000029
"(–, whatever)",0.000029
"(–, who)",0.000029
"(–, womens)",0.000029
